In [ ]:
import numpy as np

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

import time

from PyDemoMomentumopt import MotionPlanner

def replay_kinematic_plan():
    for ks in motion_planner.kin_optimizer.kinematics_sequence.kinematics_states:
        q = ks.robot_posture.generalized_joint_positions
        motion_planner.kin_optimizer.robot.display(np.matrix(q).T)
        time.sleep(0.1)

In [ ]:
# cfg_file = '../config/SAB_Demos/cfg_quadruped_forward_jump.yaml'
# cfg_file = '../config/cfg_demo01_walk_short.yaml'
cfg_file = '../config/cfg_demo08_jump.yaml'

motion_planner = MotionPlanner(cfg_file)
# optimized_kin_plan, optimized_motion_eff, optimized_dyn_plan, planner_setting, time_vector = motion_planner.optimize_motion()

In [ ]:
# Dynamic optimization
motion_planner.optimize_dynamics(0)
dynseq = motion_planner.dyn_optimizer.dynamicsSequence()

com = np.array([ds.com for ds in dynseq.dynamics_states])
plt.plot(com[:])
plt.title('COM')

motion_planner.optimize_kinematics(0)

kinstat = motion_planner.kin_optimizer.kinematics_sequence.kinematics_states

In [ ]:
motion_planner.plot_centroidal()
replay_kinematic_plan()

In [ ]:
from src.momentumopt.kinoptpy.qp import QpSolver
from pinocchio import RobotWrapper
import pinocchio as se3
from pinocchio.utils import zero

from pymomentum import \
    PlannerVectorParam_KinematicDefaultJointPositions, \
    PlannerIntParam_NumTimesteps, \
    PlannerDoubleParam_TimeStep

robot = motion_planner.kin_optimizer.robot.robot

class PointContactInverseKinematics(object):
    def __init__(self, model, endeff_frame_names):
        def getFrameId(name):
            idx = model.getFrameId(name)
            if idx == len(model.frames):
                raise Exception('Unknown frame name: {}'.format(name))
            return idx
        
        self.robot = RobotWrapper(model)
        self.model = model
        self.data = self.robot.data
        self.mass = sum([i.mass for i in robot.model.inertias])
        self.base_id = self.robot.model.getFrameId('base_link')
        self.endeff_frame_names = endeff_frame_names
        self.endeff_ids = [getFrameId(name) for name in endeff_frame_names]
        
        self.ne = len(self.endeff_ids)
        self.nv = self.model.nv
        
        # Tracking weights
        self.w_endeff_tracking = np.ones(self.ne * 3)
        self.w_com_tracking = np.ones(6)
        
        # P gains for tracking the position
        self.p_endeff_tracking = 1.
        self.p_com_tracking = 1.
        
        # Allocate space for the jacobian and desired velocities.
        # Using two entires for the linear and angular velocity of the base.
        self.J = np.zeros(((self.ne + 2) * 3, self.nv))
        self.vel_des = np.zeros(((self.ne + 2) * 3, 1))
        
        self.qp_solver = QpSolver()

    def rotate_J(self, jac, index):
        world_R_joint = se3.SE3(self.data.oMf[index].rotation, zero(3))
        return world_R_joint.action.dot(jac)
    
    def get_world_oriented_frame_jacobian(self, q, index):
        return self.rotate_J(
            se3.frameJacobian(self.model, self.data, q, index, se3.ReferenceFrame.LOCAL),
            index)
        
    def fill_jacobians(self, q):
        # REVIEW(jviereck): The Ludo invkin sets the angular momentum part to the identity.
        self.J[:6, :] = self.rotate_J(self.robot.data.Ag, self.base_id)
#         self.J[:3, :] = robot.data.Jcom * invkin.mass
        for i, idx in enumerate(self.endeff_ids):
            self.J[6 + 3 * i: 6 + 3 * (i + 1), :] = self.get_world_oriented_frame_jacobian(q, idx)[:3]
    
    def fill_vel_des(self, q, dq, com_ref, momentum_ref, endeff_pos_ref, endeff_vel_ref):
        self.vel_des[:6] = momentum_ref
        self.vel_des[:3] += self.p_com_tracking * (com_ref - self.robot.com(q).T).T
        for i, idx in enumerate(self.endeff_ids):
            self.vel_des[6 + 3*i: 6 + 3*(i + 1)] = endeff_vel_ref[i].reshape((3, 1)) + \
                self.p_endeff_tracking * (
                    endeff_pos_ref[i] - self.robot.data.oMf[idx].translation.T).T
    
    def compute(self, q, dq, com_ref, momentum_ref, endeff_pos_ref, endeff_vel_ref):
        """
        Arguments:
            q: Current robot state
            dq: Current robot velocity
            com_ref: Reference com position in global coordinates
            momentum_ref: Reference linear and angular momentum in local coordinates (?)
            endeff_pos_ref: [N_endeff x 3] Reference endeffectors position in global coordinates
            endeff_vel_ref: [N_endeff x 3] Reference endeffectors velocity in global coordinates
        """
        
        # Update the pinocchio model.
        self.robot.forwardKinematics(q, dq)
        self.robot.framesForwardKinematics(q)
        self.robot.centroidalMomentum(q, dq)
        
        
        self.fill_jacobians(q)
        self.fill_vel_des(q, dq, com_ref, momentum_ref, endeff_pos_ref, endeff_vel_ref)
        self.w = np.diag(np.hstack([self.w_com_tracking, self.w_endeff_tracking]))
        
        return self.qp_solver.quadprog_solve_qp(
            self.J.T.dot(self.w).dot(self.J), 
            -self.J.T.dot(self.w).dot(self.vel_des).reshape(-1)
        )
        
        

# Optimize the initial configuration
q = robot.model.neutralConfiguration.copy()
q[7:] = np.matrix(
    motion_planner.planner_setting.get(PlannerVectorParam_KinematicDefaultJointPositions)).T
dq = np.matrix(np.zeros(robot.nv)).T
init_state = motion_planner.ini_state

eff_names = ['{}_END'.format(name) for name in motion_planner.kin_optimizer.robot.effs]
invkin = PointContactInverseKinematics(robot.model, eff_names)

# ks = motion_planner.kin_optimizer.kinematics_sequence.kinematics_states[0]
# q = ks.robot_posture.generalized_joint_positions
# q = np.matrix(q).T
max_init_iters = 50

com_ref = init_state.com
momentum_ref = np.matrix(np.zeros(6)).T
endeff_pos_ref = np.array([init_state.effPosition(i) for i in range(init_state.effNum())])
endeff_vel_ref = np.matrix(np.zeros((init_state.effNum(), 3)))

quad_goal = se3.Quaternion(se3.rpy.rpyToMatrix(np.matrix([0.0, 0, 0]).T))

for iters in range(max_init_iters):
    # Adding small P controller for the base orientation to always start with flat
    # oriented base.
    quad_q = se3.Quaternion(float(q[6]), float(q[3]), float(q[4]), float(q[5]))
    momentum_ref[3:] = -1e-2 * se3.log((quad_q * quad_goal).matrix())
    
    res = invkin.compute(q, dq, com_ref, momentum_ref, endeff_pos_ref, endeff_vel_ref)
    q = se3.integrate(robot.model, q, np.matrix(res).T)
    
    motion_planner.kin_optimizer.robot.display(q)
    
    if np.linalg.norm(res) < 1e-5:
        print('Found initial configuration after {} iterations'.format(iters + 1))
        break
        
if iters == max_init_iters - 1:
    print('Failed to converge for initial setup.')

q_init = q.copy()
dq_init = dq.copy()


In [ ]:
%load_ext autoreload
%autoreload 2

# Build endeffector trajectory
from src.momentumopt.kinoptpy.min_jerk_traj import *

dynamic_sequence = motion_planner.dyn_optimizer.dynamicsSequence()

class Contact:

    def __init__(self, position, start_time, end_time):
        self.pos = position
        self.init_time = start_time
        self.final_time = end_time

    def position(self):
        return self.pos

    def start_time(self):
        return self.init_time

    def end_time(self):
        return self.final_time


def get_contact_plan(contact_states, effs):
    contacts = {}
    for i, eff in enumerate(effs):
        num_contacts = len(contact_states(i))
        contacts[eff] = []
        for j in range(num_contacts):
            contact_ = contact_states(i)[j]
            start_time = contact_.start_time
            end_time = contact_.end_time
            position = contact_.position
            contacts[eff].append(Contact(position, start_time, end_time))

    return contacts

contacts = get_contact_plan(motion_planner.contact_plan.contactSequence().contact_states, eff_names)

com_motion = np.array([dynamic_sequence.dynamics_states[t].com for t in range(len(dynamic_sequence.dynamics_states))])
lmom_motion = np.array([dynamic_sequence.dynamics_states[t].lmom for t in range(len(dynamic_sequence.dynamics_states))])
amom_motion = np.array([dynamic_sequence.dynamics_states[t].amom for t in range(len(dynamic_sequence.dynamics_states))])

# Generate minimum jerk trajectories
z_max = max(com_motion[:, 2])
z_min = min(com_motion[:, 2])
eff_traj_poly = generate_eff_traj(contacts, z_max, z_min)

t = np.linspace(0, 4.)

plt.plot(t, [eff_traj_poly['BR_END'][2].eval(it) for it in t])
plt.grid(True)
plt.show()

plt.plot(t, [eff_traj_poly['BR_END'][2].deval(it) for it in t])
plt.grid(True)

In [ ]:
# Create trajectories for the endeffectors

dt = motion_planner.planner_setting.get(PlannerDoubleParam_TimeStep)

num_eff = init_state.effNum()
num_time_steps = motion_planner.planner_setting.get(PlannerIntParam_NumTimesteps)

endeff_pos_ref = np.zeros((num_time_steps, num_eff, 3))
endeff_vel_ref = np.zeros((num_time_steps, num_eff, 3))

for it in range(num_time_steps):
    for eff, name in enumerate(eff_names):
        endeff_pos_ref[it][eff] = [eff_traj_poly[name][i].eval(it * dt) for i in range(3)]
        endeff_vel_ref[it][eff] = [eff_traj_poly[name][i].deval(it * dt) for i in range(3)]

In [ ]:
eff_traj_poly['BL_END'][0].deval(0)

In [ ]:
q_new = q = q_init.copy()
dq = dq_init.copy()

# momentum_ref[:3]

com_kin = np.zeros((num_time_steps, 3))
lmom_kin = np.zeros((num_time_steps, 3))
amom_kin = np.zeros((num_time_steps, 3))

invkin.w_com_tracking[:3] = 1.
invkin.w_com_tracking[3:] = 1e-12

it_opt = 12

# Run the inverse kinematics optimizer:
# for it in range(num_time_steps):
for it in range(it_opt):
    momentum_ref[:3] = np.matrix(dynamic_sequence.dynamics_states[it].lmom).T
    momentum_ref[3:] = np.matrix(dynamic_sequence.dynamics_states[it].amom).T
    
    q = q_new.copy()
    dq = invkin.compute(q, dq, com_motion[it], momentum_ref, endeff_pos_ref[it], endeff_vel_ref[it])
    dq = np.matrix(dq).T
    q_new = se3.integrate(robot.model, q, dq * dt)
    
    hg = invkin.robot.centroidalMomentum(q_new, dq)
    com_kin[it] = robot.com().T
    lmom_kin[it] = hg.linear.T
    amom_kin[it] = hg.angular.T
    
    motion_planner.kin_optimizer.robot.display(q_new)
    
    time.sleep(dt)


In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(6, 8), sharex=True)

dynseq = {'com': com_motion, 'lmom': lmom_motion, 'amom': amom_motion}
kinseq = {'com': com_kin, 'lmom': lmom_kin, 'amom': amom_kin}

for i, (ax, prop) in enumerate(zip(axes, ['com', 'lmom', 'amom'])):
    data_dyn = dynseq[prop][:it_opt]
    data_kin = kinseq[prop][:it_opt]

    for dyn, kin, label in zip(data_dyn.T, data_kin.T, ['{}_{}'.format(prop, d) for d in ['x', 'y', 'z']]):
        line = ax.plot(dyn, label=label, alpha=0.75)[0]
        ax.plot(kin, '--', color=line.get_color())[0]

    ax.legend()
    ax.grid(True)

fig.suptitle('Centroidal info for dyn (-) and kin (--)')
fig.tight_layout(rect=[0, 0, 1., 0.95])